In [2]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Load the dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# Display basic information about the dataset
print("Shape of the dataframe:", df.shape)
print("\nColumns and data types:\n", df.dtypes)
print("\nFirst few rows of the dataframe:\n", df.head())
# Preprocess the History column
def safely_eval(x):
    try:
        return eval(x)
    except:
        return []

df['History'] = df['History'].apply(lambda x: safely_eval(x) if isinstance(x, str) else x)
df['History'] = df['History'].apply(lambda x: x if isinstance(x, list) else [])
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else x)
df['ImpressionId'] = df['ImpressionId'].astype(str)

# Train Word2Vec model on the History column
history_sentences = df['History'].tolist()
w2v_model = Word2Vec(history_sentences, vector_size=100, window=5, min_count=1, sg=1, workers=4)

# Create article embeddings by averaging word embeddings
def get_article_embedding(article_ids):
    embeddings = [w2v_model.wv[news_id] for news_id in article_ids if news_id in w2v_model.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(100)

df['ArticleEmbedding'] = df['History'].apply(get_article_embedding)

# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Matrix factorization on user-item interactions
interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='History', fill_value=0)
nmf = NMF(n_components=20, random_state=42)
user_matrix = nmf.fit_transform(interaction_matrix)
item_matrix = nmf.components_

# Generate recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, df, top_n=10):
    user_index = interaction_matrix.index.tolist().index(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = [interaction_matrix.columns[idx] for idx in top_indices]
    return recommended_news_ids

# Find users with non-empty history
valid_users = train_df[train_df['History'].apply(lambda x: len(x) > 0)]
user_with_valid_history = valid_users['UserId'].iloc[0]

# Generate recommendations
recommendations = get_recommendations(user_with_valid_history, user_matrix, item_matrix, train_df)
print(f"Recommended articles for user {user_with_valid_history}:", recommendations)


Shape of the dataframe: (159957, 5)

Columns and data types:
 ImpressionId     int64
UserId          object
Time            object
History         object
Time_of_Day     object
dtype: object

First few rows of the dataframe:
    ImpressionId  UserId                 Time  \
0             1  U13740  2019-11-11 09:05:58   
1             2  U91836  2019-11-12 18:11:30   
2             3  U73700  2019-11-14 07:01:48   
3             4  U34670  2019-11-11 05:28:05   
4             5   U8125  2019-11-12 16:11:21   

                                             History Time_of_Day  
0  ['N55189', 'N42782', 'N34694', 'N45794', 'N184...     morning  
1  ['N31739', 'N6072', 'N63045', 'N23979', 'N3565...     evening  
2  ['N10732', 'N25792', 'N7563', 'N21087', 'N4108...     morning  
3  ['N45729', 'N2203', 'N871', 'N53880', 'N41375'...       night  
4           ['N10078', 'N56514', 'N14904', 'N33740']   afternoon  


TypeError: complex() first argument must be a string or a number, not 'list'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF

# Load your dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# No need to apply eval() as History already contains lists
# You might want to strip 'N' from news IDs if necessary:
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else [])

# Simulate uplift labels (example: randomly assign uplift scores)
df['UpliftScore'] = np.random.uniform(0, 1, df.shape[0])

# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Matrix factorization on user-item interactions or uplift scores
interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)
nmf = NMF(n_components=20, random_state=42)
user_matrix = nmf.fit_transform(interaction_matrix)
item_matrix = nmf.components_

# Generate recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, df, top_n=10):
    user_index = interaction_matrix.index.tolist().index(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = [interaction_matrix.columns[idx] for idx in top_indices]
    return recommended_news_ids

# Example: Get recommendations for a user
user_id = train_df['UserId'].iloc[0]
recommendations = get_recommendations(user_id, user_matrix, item_matrix, train_df)
print(f"Recommended articles for user {user_id}:", recommendations)

<ipython-input-2-72d2f08f6c9b>:22: PerformanceWarning: The following operation may generate 5817831652 cells in the resulting pandas object.
  interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)


In [1]:
from scipy.sparse import csr_matrix

interaction_matrix_sparse = csr_matrix(interaction_matrix)
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# Load your dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# No need to apply eval() as History already contains lists
# You might want to strip 'N' from news IDs if necessary:
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else [])

# Simulate uplift labels (example: randomly assign uplift scores)
df['UpliftScore'] = np.random.uniform(0, 1, df.shape[0])

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Matrix factorization on user-item interactions or uplift scores
interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)
interaction_matrix_sparse = csr_matrix(interaction_matrix)  # Convert to sparse matrix

# Reduce n_components based on your memory constraints
nmf = NMF(n_components=10, random_state=42)  # Adjust n_components as needed
user_matrix = nmf.fit_transform(interaction_matrix_sparse)
item_matrix = nmf.components_

# Generate recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix, top_n=10):
    user_index = interaction_matrix.index.get_loc(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = interaction_matrix.columns[top_indices].tolist()
    return recommended_news_ids

# Example: Get recommendations for a user
user_id = train_df['UserId'].iloc[0]
recommendations = get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix)
print(f"Recommended articles for user {user_id}:", recommendations)


NameError: name 'interaction_matrix' is not defined

In [1]:
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# Load your dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# No need to apply eval() as History already contains lists
# You might want to strip 'N' from news IDs if necessary:
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else [])

# Simulate uplift labels (example: randomly assign uplift scores)
df['UpliftScore'] = np.random.uniform(0, 1, df.shape[0])

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)



In [ ]:
# Matrix factorization on user-item interactions or uplift scores
interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0) # Define interaction_matrix here

interaction_matrix_sparse = csr_matrix(interaction_matrix)  # Convert to sparse matrix

# Reduce n_components based on your memory constraints
nmf = NMF(n_components=10, random_state=42)  # Adjust n_components as needed
user_matrix = nmf.fit_transform(interaction_matrix_sparse)
item_matrix = nmf.components_


<ipython-input-2-99b44b883cdf>:2: PerformanceWarning: The following operation may generate 5817831652 cells in the resulting pandas object.
  interaction_matrix = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0) # Define interaction_matrix here


In [ ]:

# Generate recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix, top_n=10):
    user_index = interaction_matrix.index.get_loc(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = interaction_matrix.columns[top_indices].tolist()
    return recommended_news_ids

# Example: Get recommendations for a user
user_id = train_df['UserId'].iloc[0]
recommendations = get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix)
print(f"Recommended articles for user {user_id}:", recommendations)

In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# Load your dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# No need to apply eval() as History already contains lists
# You might want to strip 'N' from news IDs if necessary:
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else [])

# Simulate uplift labels (example: randomly assign uplift scores)
df['UpliftScore'] = np.random.uniform(0, 1, df.shape[0])

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Example of batch processing for matrix factorization
chunk_size = 1000  # Define the size of each batch
n_components = 10  # Define the number of components for NMF

# Initialize empty matrices to store results
user_matrix = None
item_matrix = None

# Process data in chunks
for chunk_start in range(0, len(train_df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(train_df))
    chunk_df = train_df.iloc[chunk_start:chunk_end]

    # Matrix factorization on user-item interactions or uplift scores for the chunk
    interaction_matrix = chunk_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)
    interaction_matrix_sparse = csr_matrix(interaction_matrix)

    nmf = NMF(n_components=n_components, random_state=42)
    user_chunk_matrix = nmf.fit_transform(interaction_matrix_sparse)
    item_chunk_matrix = nmf.components_

    # Aggregate results from each chunk
    if user_matrix is None:
        user_matrix = user_chunk_matrix
        item_matrix = item_chunk_matrix
    else:
        user_matrix = np.vstack((user_matrix, user_chunk_matrix))
        item_matrix += item_chunk_matrix

# Finalize item_matrix by averaging across chunks
item_matrix /= len(train_df) / chunk_size

# Example: Get recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix, top_n=10):
    user_index = interaction_matrix.index.get_loc(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = interaction_matrix.columns[top_indices].tolist()
    return recommended_news_ids

# Example: Get recommendations for a user
user_id = train_df['UserId'].iloc[0]
recommendations = get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix)
print(f"Recommended articles for user {user_id}:", recommendations)


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea

ValueError: operands could not be broadcast together with shapes (10,1000) (10,999) (10,1000) 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split

# Load your dataset
data_path = '/content/sampled_processed.csv'
df = pd.read_csv(data_path)

# No need to apply eval() as History already contains lists
# You might want to strip 'N' from news IDs if necessary:
df['History'] = df['History'].apply(lambda x: [news_id.lstrip('N') for news_id in x] if isinstance(x, list) else [])

# Simulate uplift labels (example: randomly assign uplift scores)
df['UpliftScore'] = np.random.uniform(0, 1, df.shape[0])

# Split the dataset into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Example of batch processing for matrix factorization
chunk_size = 1000  # Define the size of each batch
n_components = 10  # Define the number of components for NMF

# Initialize empty lists to store results
user_matrices = []
item_matrices = []

# Process data in chunks
for chunk_start in range(0, len(train_df), chunk_size):
    chunk_end = min(chunk_start + chunk_size, len(train_df))
    chunk_df = train_df.iloc[chunk_start:chunk_end]

    # Matrix factorization on user-item interactions or uplift scores for the chunk
    interaction_matrix = chunk_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)
    interaction_matrix_sparse = csr_matrix(interaction_matrix)

    nmf = NMF(n_components=n_components, random_state=42)
    user_chunk_matrix = nmf.fit_transform(interaction_matrix_sparse)
    item_chunk_matrix = nmf.components_

    # Append results to lists
    user_matrices.append(user_chunk_matrix)
    item_matrices.append(item_chunk_matrix)

# Concatenate user matrices vertically
user_matrix = np.vstack(user_matrices)

# Pad item matrices to ensure consistent shapes before averaging
max_n_items = max([item_matrix.shape[1] for item_matrix in item_matrices])
padded_item_matrices = [np.pad(item_matrix, ((0, 0), (0, max_n_items - item_matrix.shape[1])), 'constant')
                       for item_matrix in item_matrices]

# Calculate the average of item matrices after padding
item_matrix = np.mean(padded_item_matrices, axis=0)

# Example: Get recommendations for a user
def get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix, top_n=10):
    user_index = interaction_matrix.index.get_loc(user_id)
    user_vector = user_matrix[user_index]
    scores = np.dot(user_vector, item_matrix)
    top_indices = np.argsort(scores)[::-1][:top_n]
    recommended_news_ids = interaction_matrix.columns[top_indices].tolist()
    return recommended_news_ids

# Example: Get recommendations for a user
# Note: You may need to adjust the interaction_matrix used here depending on
# how you want to generate recommendations after batch processing.
# Build a single interaction matrix from the entire training set
interaction_matrix_all = train_df.pivot_table(index='UserId', columns='ImpressionId', values='UpliftScore', fill_value=0)

user_id = train_df['UserId'].iloc[0]
recommendations = get_recommendations(user_id, user_matrix, item_matrix, interaction_matrix_all) # Use interaction_matrix_all
print(f"Recommended articles for user {user_id}:", recommendations)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklea